In [1]:
# Mistral-7B needs latest transformers library, so upgrade with self built wheel from latest source!
!pip install --no-index --no-deps /kaggle/input/mistral-7b-v0-1-starter/transformers-4.34.0.dev0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/transformers-434dev/tokenizers-0.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/mistral-7b-v0-1-starter/transformers-4.34.0.dev0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Processing /kaggle/input/transformers-434dev/tokenizers-0.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3


In [2]:
import sys
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
# model_name = '/kaggle/input/mistral-7b-v0-1-starter/Mistral-7B-v0.1/Mistral-7B-v0.1'
model_name = '/kaggle/input/mistral-7b-instruct-v0-1/Mistral-7B-Instruct-v0.1'


tokenizer = AutoTokenizer.from_pretrained(model_name)
    
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-pack

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# prompt = "Translate the following sentence from English to Hindi:\n query\n Translated Sentence: "
prompt = "Translate this to Hindi from English\nText: engSen\nTranslated Text: "


inputs = tokenizer(prompt.replace('engSen', "Its facinating to see how much the city have advance with lots of cars, smartphones and huge buildings.") , return_tensors="pt").to(f"cuda:{model.device.index}")
print('input')
print(input)
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                            return_dict_in_generate=True, output_scores=True,pad_token_id=tokenizer.eos_token_id)
# Print the translated sentence
print(output.sequences[0])

# Decode the translated sentence
translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Print the translated sentence
print(translated_sentence)

input
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7803e44f6560>>
tensor([    1,  4335, 10020,   456,   298, 14526, 28710,   477,  4300,    13,
         1874, 28747,  7267,  3252, 11251,   298,  1032,   910,  1188,   272,
         2990,   506,  8670,   395,  8532,   302,  8300, 28725,  7455, 18624,
          304,  5401,  9702, 28723,    13,  3131, 28714,   601,  7379, 28747,
        28705, 31691, 29708, 29300, 29794, 29232, 30046, 29540, 28705, 29499,
        29586, 28705, 29759, 29554, 31014, 28705, 29972, 29554, 29929, 30046,
        29735, 28705, 29672, 29586, 29929, 28705, 30318, 29300, 29319, 29554,
        29896, 30003, 29929, 28705, 29499, 30003, 28705, 29759, 29586, 29499,
        29554, 29540, 28705, 31691, 29540, 29300, 29972, 30003, 29929, 29499,
        30003, 28705, 30766, 29794, 29540, 29586, 28705, 31691, 29708, 29300,
        29794, 29232, 30046, 28705, 29499, 29586, 28705, 29759, 29554, 31014,
        28705, 31691, 29708, 29300, 29794

In [6]:
prompt = "Translate this to Hindi from English\nText: I've never seen it\nTranslated Text: maine to kabhi nahi dekha hai.\n\n Translate this to Hindi from English\nText: I like Robert Downey too\nTranslated Text: Mujhe bhi Robert Downey acha lagta hai\n\n Translate this to Hindi from English\nText: engSen\nTranslated Text: "
# prompt = "Translate this to Hindi Roman from English\nText: engSen\nTranslated Text: "

inputs = tokenizer(prompt.replace('engSen', "I am a proud stuednt of IIIT Hyderabad.") , return_tensors="pt").to(f"cuda:{model.device.index}")
print('input')
print(input)
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                            return_dict_in_generate=True, output_scores=True,pad_token_id=tokenizer.eos_token_id)
# Print the translated sentence
print(output.sequences[0])

# Decode the translated sentence
translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Print the translated sentence
print(translated_sentence)

input
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7803e44f6560>>
tensor([    1,  4335, 10020,   456,   298, 14526, 28710,   477,  4300,    13,
         1874, 28747,   315, 28742,   333,  1484,  2598,   378,    13,  3131,
        28714,   601,  7379, 28747,  2191, 28706,   298,   446,   375,  5365,
        20330, 28710,   340, 28729,  2799,   295,  1585, 28723,    13,    13,
         4335, 10020,   456,   298, 14526, 28710,   477,  4300,    13,  1874,
        28747,   315,   737,  5606, 20271,  2705,  1368,    13,  3131, 28714,
          601,  7379, 28747,   351,  5872,   265,   287,  5365,  5606, 20271,
         2705,   264,  8136, 16369,  1100,   295,  1585,    13,    13,  4335,
        10020,   456,   298, 14526, 28710,   477,  4300,    13,  1874, 28747,
          315,   837,   264,  9393,   341,  2465,   448,   302,  3717,  1153,
        10540,   826, 20882, 28723,    13,  3131, 28714,   601,  7379, 28747,
        28705,   351,  5872,   265,  3717

In [7]:
prompt = "Translate this to Hinglish from English\nText: I've never seen it\nTranslated Text: maine to kabhi nahi dekha hai.\n\n Translate this to Hinglish from English\nText: I like Robert Downey too\nTranslated Text: Mujhe bhi Robert Downey acha lagta hai\n\n Translate this to Hinglish from English\nText: engSen\nTranslated Text: "
# prompt = "Translate this to Hindi Roman from English\nText: engSen\nTranslated Text: "

inputs = tokenizer(prompt.replace('engSen', "I am a proud stuednt of IIIT Hyderabad.") , return_tensors="pt").to(f"cuda:{model.device.index}")
print('input')
print(input)
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                            return_dict_in_generate=True, output_scores=True,pad_token_id=tokenizer.eos_token_id)
# Print the translated sentence
print(output.sequences[0])

# Decode the translated sentence
translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Print the translated sentence
print(translated_sentence)

input
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7803e44f6560>>
tensor([    1,  4335, 10020,   456,   298,   382,   288,  2295,   477,  4300,
           13,  1874, 28747,   315, 28742,   333,  1484,  2598,   378,    13,
         3131, 28714,   601,  7379, 28747,  2191, 28706,   298,   446,   375,
         5365, 20330, 28710,   340, 28729,  2799,   295,  1585, 28723,    13,
           13,  4335, 10020,   456,   298,   382,   288,  2295,   477,  4300,
           13,  1874, 28747,   315,   737,  5606, 20271,  2705,  1368,    13,
         3131, 28714,   601,  7379, 28747,   351,  5872,   265,   287,  5365,
         5606, 20271,  2705,   264,  8136, 16369,  1100,   295,  1585,    13,
           13,  4335, 10020,   456,   298,   382,   288,  2295,   477,  4300,
           13,  1874, 28747,   315,   837,   264,  9393,   341,  2465,   448,
          302,  3717,  1153, 10540,   826, 20882, 28723,    13,  3131, 28714,
          601,  7379, 28747, 28705,   351

In [8]:
prompt = "Translate this to Hinglish from English\nText: engSen\nTranslated Text: "

inputs = tokenizer(prompt.replace('engSen', "Thats a sexy car") , return_tensors="pt").to(f"cuda:{model.device.index}")
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                            return_dict_in_generate=True, output_scores=True,pad_token_id=tokenizer.eos_token_id)

# Decode the translated sentence
translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Print the translated sentence
print(translated_sentence)

Translate this to Hinglish from English
Text: Thats a sexy car
Translated Text: ðæt's æ sexy cær


In [9]:
prompt = "What is it called when Hindi is used with mix English words "

inputs = tokenizer(prompt , return_tensors="pt").to(f"cuda:{model.device.index}")
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                            return_dict_in_generate=True, output_scores=True,pad_token_id=tokenizer.eos_token_id)

# Decode the translated sentence
translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Print the translated sentence
print(translated_sentence)

What is it called when Hindi is used with mix English words 

## Answer (1)

It is called **Hinglish**.

> Hinglish is a term used to describe a variety of English that is influenced by Hindi. It is a blend of English and Hindi, and is spoken by many people in India and other parts of the world.

Source: https://en.wikipedia.org/wiki/Hinglish


In [11]:
train_files = ["/kaggle/input/enghin-hinglish/train.txt", "/kaggle/input/enghin-hinglish/train_roman.txt", "/kaggle/input/enghin-hinglish/train_deva.txt"]
for txt in train_files:
    print(txt[30:-4] + "ZeroShot.txt")

trainZeroShot.txt
train_romanZeroShot.txt
train_devaZeroShot.txt


In [12]:
import random

train_files = ["/kaggle/input/enghin-hinglish/train_deva.txt"]
test_files = ["/kaggle/input/enghin-hinglish/test_deva.txt"]
dev_files = ["/kaggle/input/enghin-hinglish/dev_deva.txt"]

# Function to extract the first sentence from a line
def extract_first_sentence(line):
    return line.split('\t')[0]

# Function for 3 Shot Prompting
def three_shot_prompt(input_sentence, model, tokenizer, idx):
        
    file_path = train_files[idx]
    
    # Read the dataset file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    selected_indices = random.sample(range(len(lines)-1), 3)

    line1 = lines[selected_indices[0]].strip().split('\t')[0]
    line2 = lines[selected_indices[0]].strip().split('\t')[1]
    line3 = lines[selected_indices[1]].strip().split('\t')[0]
    line4 = lines[selected_indices[1]].strip().split('\t')[1]
    line5 = lines[selected_indices[2]].strip().split('\t')[0]
    line6 = lines[selected_indices[2]].strip().split('\t')[1]
    
    prompt = f"----------\nTranslate this to Hinglish from English\nText: {line1}\nTranslated Text: <SOS> {line2} <EOS>\n####\nTranslate this to Hinglish from English\nText: {line3}\nTranslated Text: <SOS> {line4} <EOS>\n####\nTranslate this to Hinglish from English\nText: {line5}\nTranslated Text: <SOS> {line6} <EOS>\n####\nTranslate this to Hinglish from English\nText: {input_sentence}\nTranslated Text:"

    inputs = tokenizer(prompt, return_tensors="pt").to(f"cuda:{model.device.index}")
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100, 
                                return_dict_in_generate=True, output_scores=True, pad_token_id=tokenizer.eos_token_id)

    translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

    return translated_sentence

# Process each file
idx = 0
for trainFile, testFile, devFile in zip(train_files, test_files, dev_files):
    # Read the dataset file
    with open(devFile, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    output_file = devFile[30:-4] + "_ThreeShotPrompting.txt"
    print(output_file)

    # Open the output file for writing
    with open(output_file, 'w', encoding='utf-8') as output_file_handle:
        for line in lines:
            # Split the line into InputSent and TrueAns
            input_sent, true_ans = line.strip().split('\t')

            # Perform 3 Shot Prompting
            translated_sentence = three_shot_prompt(input_sent, model, tokenizer, idx)

            # Write the translated sentence along with the TrueAns to the output file
            output_file_handle.write(f"{translated_sentence}\n")
            
            print(translated_sentence)
    idx += 1
    print(f"Translations for {trainFile} are saved in {output_file}")

dev_deva_ThreeShotPrompting.txt
----------
Translate this to Hinglish from English
Text: क्या ड्रैगन बोलता है?
Translated Text: <SOS> Kya dragon baat kar sakte hai? <EOS>
####
Translate this to Hinglish from English
Text: मुझे बस यह कहने दो कि वह दूर हो जाता है और मिसाइल का निर्माण नहीं करता है।
Translated Text: <SOS> Main bas ye kehta hu k wo bhag jata hai aur missile nahi banata hai <EOS>
####
Translate this to Hinglish from English
Text: एक बच्चों की फिल्म पर 79% मेरी राय में महत्वपूर्ण दृश्यों को पढ़ने के बाद बहुत बुरा नहीं लगता है।
Translated Text: <SOS> 79% kids movie tho bad hi hota mera opinion se, keyscenes ko badne ke baath <EOS>
####
Translate this to Hinglish from English
Text: हैलो
Translated Text: <SOS> Hello <EOS>
####
Translate this to Hinglish from English
Text: क्या आप किता किता किता किता किता किता किता किता किता किता किता किता किता कि
----------
Translate this to Hinglish from English
Text: इसके लिए आलोचनात्मक प्रतिक्रियाएं भयानक थीं
Translated Text: <SOS> critical r

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:57                                                                                   │
│                                                                                                  │
│   54 │   │   │   input_sent, true_ans = line.strip().split('\t')                                 │
│   55 │   │   │                                                                                   │
│   56 │   │   │   # Perform 3 Shot Prompting                                                      │
│ ❱ 57 │   │   │   translated_sentence = three_shot_prompt(input_sent, model, tokenizer, idx)      │
│   58 │   │   │                                                                                   │
│   59 │   │   │   # Write the translated sentence along with the TrueAns to the output file       │
│   60 │   │   │   output_file_handle.write(f"{translated_sentence}\n")                            │
│                                                                                                  │
│ in three_shot_prompt:33                                                                          │
│                                                                                                  │
│   30 │                                                                                           │
│   31 │   inputs = tokenizer(prompt, return_tensors="pt").to(f"cuda:{model.device.index}")        │
│   32 │   with torch.no_grad():                                                                   │
│ ❱ 33 │   │   output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["at    │
│   34 │   │   │   │   │   │   │   │   return_dict_in_generate=True, output_scores=True, pad_to    │
│   35 │                                                                                           │
│   36 │   translated_sentence = tokenizer.decode(output.sequences[0], skip_special_tokens=True    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1606 in generate        │
│                                                                                                  │
│   1603 │   │   │   )                                                                             │
│   1604 │   │   if generation_mode == GenerationMode.GREEDY_SEARCH:                               │
│   1605 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1606 │   │   │   return self.greedy_search(                                                    │
│   1607 │   │   │   │   input_ids,                                                                │
│   1608 │   │   │   │   logits_processor=logits_processor,                                        │
│   1609 │   │   │   │   stopping_criteria=stopping_criteria,